In [1]:
import cv2 
import mediapipe as mp

BaseOptions = mp.tasks.BaseOptions
ImageEmbedder = mp.tasks.vision.ImageEmbedder
ImageEmbedderOptions = mp.tasks.vision.ImageEmbedderOptions
VisionRunningMode = mp.tasks.vision.RunningMode

In [2]:

def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized


In [13]:

BaseOptions = mp.tasks.BaseOptions
ImageEmbedder = mp.tasks.vision.ImageEmbedder
ImageEmbedderOptions = mp.tasks.vision.ImageEmbedderOptions
VisionRunningMode = mp.tasks.vision.RunningMode

new_options = ImageEmbedderOptions(
    base_options=BaseOptions(model_asset_path='mobilenet_v3_large.tflite'),
    l2_normalize=True,
    quantize=True,
    running_mode=VisionRunningMode.IMAGE)

embedder=ImageEmbedder.create_from_options(new_options)
def get_embadding(img):
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
    embedding_result = embedder.embed(mp_image)
    embedding_list = embedding_result.embeddings[0].embedding.tolist()
    return embedding_list

import csv
import os
def save_face_data_to_csv(face_data, ids, filename):
    if not isinstance(face_data, list):
        print("Error: face_data must be a list.")
        return
    file_exists = os.path.exists(filename)
    with open(filename, 'a' if file_exists else 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        if not file_exists:
            writer.writerow(["face_data", "id"])
        
        writer.writerow([face_data, ids])
    if file_exists:
        pass 
        # print(f"Face data appended to {filename} successfully.")
    else:
        pass
        # print(f"Face data saved to {filename} successfully.")
    

In [14]:
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

In [15]:
base_options = python.BaseOptions(model_asset_path='blaze_face_short_range.tflite')
options = vision.FaceDetectorOptions(base_options=base_options)
detector = vision.FaceDetector.create_from_options(options)

In [18]:
def extract_facial_data(IMAGE_FILE):
    img = cv2.imread(IMAGE_FILE)
    image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
    img = cv2.imread(IMAGE_FILE)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    detection_result = detector.detect(image)
    if detection_result.detections:
        for detection in detection_result.detections:
            bbox = detection.bounding_box
            x1, y1 = int(bbox.origin_x),int( bbox.origin_y)
            x2, y2 = int(bbox.origin_x + bbox.width), int(bbox.origin_y + bbox.height)
            x1, y1 = x1-20, y1-170
            x2, y2 = x2+30, y2+130
            # cv2.rectangle (img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            crop_img = img[y1:y2, x1:x2]
            crop_img = image_resize(crop_img, width=480, height=480)
            crop_img = cv2.cvtColor(crop_img, cv2.COLOR_RGB2BGR)
            face_data=get_embadding(crop_img) 
            save_face_data_to_csv(face_data, 1, "data.csv")

In [9]:
import glob

def get_image_paths(directory):
    image_paths = []
    extensions = ["jpg", "jpeg", "png"]
    for ext in extensions:
        image_paths.extend(glob.glob(directory + "/*." + ext))
    return image_paths

directory = "./images/"
image_paths = get_image_paths(directory)

for path in image_paths:
    try:
        extract_facial_data(path)
    except:
        pass

